Code Description
This cell sets up the required libraries and configurations for the analysis:

Imports:

config: Likely contains custom settings or credentials, such as database connection strings.
pandas: A powerful library for data manipulation, allowing for easy handling of the tabular data used in this analysis.
matplotlib.pyplot: A plotting library for creating visualizations, which will be useful for representing geographical data.
FixedLocator: A specific tool from matplotlib to fix tick locations in plots, ensuring consistent visual representation.
sqlalchemy.create_engine: Enables connection to databases (likely a PostGIS or another spatial database) to retrieve and manage geospatial data.
Pandas Configuration:

The display precision is set to 2 decimal places for floating-point numbers. This ensures consistent representation of numerical data, such as slope angles or solar irradiance values, in the subsequent DataFrame outputs.
This setup is essential for the further steps in the analysis, where data is retrieved, manipulated, and visualized.

In [227]:
# Import necessary libraries and modules
import config
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import FixedLocator
from sqlalchemy import create_engine

# Set display option for pandas DataFrames to limit float precision to 2 decimal places
pd.set_option("display.precision", 2)

Database Connection Setup
Configuration Data:

The mysql dictionary is imported from the config module, which stores database credentials such as the username, password, host, port, and database name.
Connection String:

A PostgreSQL connection string is constructed using the credentials from the mysql dictionary.
The connection string follows the format:
postgresql://<user>:<password>@<host>:<port>/<dbname>
Database Engine:

The create_engine() function from SQLAlchemy is used to create a connection engine, which allows communication with the PostgreSQL database. This will enable efficient querying and manipulation of data stored in the database.
Connection:

The conn object is created by calling engine.connect(), which establishes an active connection to the database. This connection can now be used to perform SQL queries and retrieve the required data for analysis.
This step is critical as it sets up the database connection, allowing access to the data needed for analyzing potential wind and solar farm locations.

In [ ]:
# Import database configuration
mysql = config.mysql

# Construct PostgreSQL connection string
connection_string = (
    f"postgresql://{mysql['user']}:{mysql['password']}@"
    f"{mysql['host']}:{mysql['port']}/{mysql['dbname']}"
)

# Create database engine
engine = create_engine(connection_string)

Defining Polish Provinces
This section defines a list of Polish provinces that will be used for data filtering or selection purposes during the analysis. Each province represents a geographical region in Poland, where the potential locations for wind and solar farms will be evaluated.

The list includes the names of all 16 provinces (województwa) in Poland:

Dolnośląskie
Kujawsko-pomorskie
Lubelskie
Lubuskie
Łódzkie
Małopolskie
Mazowieckie
Opolskie
Podkarpackie
Podlaskie
Pomorskie
Śląskie
Świętokrzyskie
Warmińsko-mazurskie
Wielkopolskie
Zachodniopomorskie

In [228]:
# Define a list of Polish provinces (województwa)
provinces = [
    "dolnoslaskie",
    "kujawsko_pomorskie",
    "lubelskie",
    "lubuskie",
    "lodzkie",
    "malopolskie",
    "mazowieckie",
    "opolskie",
    "podkarpackie",
    "podlaskie",
    "pomorskie",
    "slaskie",
    "swietokrzyskie",
    "warminsko_mazurskie",
    "wielkopolskie",
    "zachodniopomorskie"
]


Renewable Energy Potential by Province
This section calculates the total potential area available for solar and wind energy production across Polish provinces. It retrieves data from two types of tables:

Solar photovoltaic potential area: Calculated from solar_radiation_photovoltaic_area_{province} tables.
Wind energy potential area: Calculated from wind_speed_area_{province} tables.
The data for each province is fetched from a PostgreSQL database, and the total area for both solar and wind potential is computed. The result is presented in square kilometers (km²) for easier interpretation.

Steps:
Data Initialization:

Two dictionaries, solar_area_append_total and wind_area_append_total, are created to store the results for each province.
Data Extraction:

For each province, the corresponding table names for solar and wind energy data are generated.
SQL queries are constructed to sum up the available area (SUM(area)) for both solar and wind in each province.
Data Validation:

After fetching data using pd.read_sql, it is checked whether valid data was retrieved. If no data is found or the area is None, the area is set to 0.
Unit Conversion:

The area is converted from square meters (m²) to square kilometers (km²) using a factor of 1e-6, and the result is rounded to 2 decimal places.
Data Storage:

The calculated values are appended to the dictionaries, which are later converted into Pandas DataFrames for both solar and wind areas.
The resulting DataFrames, df_solar and df_wind, hold the calculated total potential area for renewable energy in each province.

In [230]:
# Initialize dictionaries to store total renewable areas for solar and wind by province
solar_area_append_total = {'Province': [], 'solar_area': []}
wind_area_append_total = {'Province': [], 'wind_area': []}

# Establish persistent connection to the database
conn = engine.connect()

# Import data from the database for each province
for province in provinces:
    
    # Define table names for solar and wind data
    table_name_solar = f"solar_radiation_photovoltaic_area_{province}"
    table_name_wind = f"wind_speed_area_{province}"
  
    # SQL queries to get the total area for solar and wind energy
    query_solar = (f"SELECT SUM(area) as solar_total_area FROM {table_name_solar}")
    query_wind = (f"SELECT SUM(area) as wind_total_area FROM {table_name_wind}")
    
    # Execute queries and store results in pandas DataFrames
    solar_result = pd.read_sql(query_solar, conn)
    wind_result = pd.read_sql(query_wind, conn)
    
    # Check if valid data is retrieved, set area to 0 if data is missing
    solar_area = solar_result['solar_total_area'][0] if not solar_result.empty and solar_result['solar_total_area'][0] is not None else 0
    wind_area = wind_result['wind_total_area'][0] if not wind_result.empty and wind_result['wind_total_area'][0] is not None else 0
    
    # Append results to the respective lists, converting m² to km²
    solar_area_append_total['Province'].append(province)
    solar_area_append_total['solar_area'].append(round(solar_area * 1e-6, 2))
    
    wind_area_append_total['Province'].append(province)
    wind_area_append_total['wind_area'].append(round(wind_area * 1e-6, 2))

# Close the database connection after the loop
conn.close()

# Convert the dictionaries to pandas DataFrames
df_solar = pd.DataFrame(solar_area_append_total)
df_wind = pd.DataFrame(wind_area_append_total)

# Display the solar potential area DataFrame
df_solar

,Province,solar_area
0,dolnoslaskie,480.64
1,kujawsko_pomorskie,262.50
2,lubelskie,594.29
3,lubuskie,326.41
4,lodzkie,485.20
5,malopolskie,441.87
6,mazowieckie,1344.64
7,opolskie,90.20
8,podkarpackie,568.48
9,podlaskie,962.12


## 5. Bar plot with solar area for each province

In [ ]:
# Bar plot
plt.figure(figsize=(14, 7))
width = 0.35  # szerokość słupków

# Pozycje słupków dla każdego województwa
ind = range(len(df_solar['Province']))

# Tworzenie słupków dla energii słonecznej i wiatrowej
plt.bar(ind, df_solar['solar_area'], width=width, label='Solar Area', color='blue')
plt.bar([i + width for i in ind], df_wind['wind_area'], width=width, label='Wind Area', color='green')

# Dodanie opisów i tytułu
plt.xlabel('Province')
plt.ylabel('Total Area (km²)')
plt.title('Comparison of Solar and Wind Energy Area by Province')
plt.xticks([i + width / 2 for i in ind], df_solar['Province'], rotation=90)
plt.legend()

# Wyświetlenie wykresu
plt.tight_layout()
plt.show()

## 6. Parametry paneli fotowoltaicznych jako zmienne, które mozna dostosować

In [ ]:
#photovoltaic standard panels parametrs [m^2]
height_panel = 1.016
width_panel = 1.686
power_output = 0.350 # unit [kW]
power_output_m2 = power_output / (height_panel * width_panel)

#panel efficiency by producent 25*C, 1000 [Wm^2]/ power efficency ~20%
efficiency = (power_output / (height_panel*width_panel  * 1))

#create effective area. We can't use whole area. In this case we use 80% area to solar panels and 20% for equipment and maintenance access
#parametrs 
solar_maintenance_access = 0.2
effective_solar_panel_area = 1 - solar_maintenance_access

#calculate the effective area
df_solar['solar_effective_area'] = (df_solar['solar_area'] * effective_solar_panel_area)
df_solar


## 7. Zebranie ilości produkowanej energii dla każdego miesiąca

In [ ]:
#select tables
months = {
    "january" : 31
    , "february" : 28
    , "march" : 31
    , "april" : 30
    ,"may" : 31
    , "june" : 30
    , "july" : 31
    , "august" : 31
    ,"september" : 30
    , "october" : 31
    , "november" : 30
    , "december" : 31
}

#solar radiation by province and each month
solar_radiation = pd.DataFrame(index=provinces, columns=months.keys())

for province in provinces:
    for month in months:
        table_name_solar = f"solar_radiation_photovoltaic_area_{province}"
        # select data in tables
        #sum of multipe area in m^2 * amount of solar radiation in month [kW] * effective area (80%) * efficency 
        query = f"SELECT SUM(area * {month} * {effective_solar_panel_area} * {efficiency}) as total FROM {table_name_solar}"

        # result to pandas
        result_df = pd.read_sql(query, conn)
        monthly_solar_energy = result_df.iloc[0]['total'] if not result_df.empty else 0
        #solar panel and change kW to  mw
        solar_radiation.at[province, month] = round(monthly_solar_energy * 1e-3,2)
        
#rename index        
solar_radiation.reset_index(inplace=True)
solar_radiation.rename(columns={'index': 'Province'}, inplace=True)

#merge df_solar and solar_radiation
df_solar = df_solar.merge(solar_radiation, left_on='Province', right_on='Province')


In [ ]:
df_wind

## 8. Zdefiniowanie parametrów dla turbin wiatrowych

In [ ]:
#functional assumption 3 wind turbine for km^2
turbine_number_for_area = 3
df_wind['turbines_number'] = df_wind['wind_area'] * turbine_number_for_area
df_wind['turbines_number'] = df_wind['turbines_number'].astype(int)
#parametrs for wind turbin Vestas V100

#minimum wind speed [m/s]
minimum_wind_speed = 3

#maximum wind speed [m/s]
maximum_wind_speed = 22

#energy generate for 1 hour when wind speed is 22 [mW]
maximum_power_generate = 2 


## 9. Wybranie z bazy danych obszarów spełniających warunki budowy farmy wiatrowej

In [ ]:
wind_energy = pd.DataFrame(index=provinces, columns=months.keys())

for province in provinces:
    for month,days in months.items():
        table_name_wind = f"wind_speed_area_{province}"
        
        # select data in tables
        #energy generate for 1 month [mW]
        query = (f"SELECT SUM(CASE "
         f"WHEN {month} > {minimum_wind_speed} THEN ((({month} * 24 * {days}) / {maximum_wind_speed}) * {maximum_power_generate})"
         f"ELSE 0 "
         f"END)  as total "
         f"FROM wind_speed_area_{province}")

        # result to pandas
        result_df = pd.read_sql(query, conn)
        monthly_wind_speed = result_df.iloc[0]['total'] if not result_df.empty else 0
        
    
        #change mW 
        monthly_wind_speed = round(monthly_wind_speed , 2)
        wind_energy.at[province, month] = monthly_wind_speed
        
#rename index        
wind_energy.reset_index(inplace=True)
wind_energy.rename(columns={'index': 'Province'}, inplace=True)

#merge df_solar and solar_radiation
df_wind = df_wind.merge(wind_energy, left_on='Province', right_on='Province')

df_wind

## 10. Obliczenie kosztow budowy produkcji farmy fotowoltaicznej na podstawie dostęnych danych

In [ ]:
solar_radiation_photovoltaic_area_by_each_province = {}
for province in provinces:
    
    # Analiza kosztów dla pojedynczych województw
    table_name = f"solar_radiation_photovoltaic_area_{province}"
    
    # Definiowanie zapytania SQL
    sum_columns = ' + '.join(months)
    
    query = (
        f"SELECT "
        f"id, "
        f"area * 1e-6 AS area_km2, "
        f"line_dista * 1e-3 AS line_distance_km, "
        f"road_dista * 1e-3 AS road_distance_km, "
        f"area * {effective_solar_panel_area} * {efficiency} * 1e-3 AS solar_energy_production_mwh, "
    
        # Miesięczna produkcja energii
        f"area * {effective_solar_panel_area} * {efficiency} * january * 1e-3 AS january_solar_energy_production_mwh, "
        f"area * {effective_solar_panel_area} * {efficiency} * february * 1e-3 AS february_solar_energy_production_mwh, "
        f"area * {effective_solar_panel_area} * {efficiency} * march * 1e-3 AS march_solar_energy_production_mwh, "
        f"area * {effective_solar_panel_area} * {efficiency} * april * 1e-3 AS april_solar_energy_production_mwh, "
        f"area * {effective_solar_panel_area} * {efficiency} * may * 1e-3 AS may_solar_energy_production_mwh, "
        f"area * {effective_solar_panel_area} * {efficiency} * june * 1e-3 AS june_solar_energy_production_mwh, "
        f"area * {effective_solar_panel_area} * {efficiency} * july * 1e-3 AS july_solar_energy_production_mwh, "
        f"area * {effective_solar_panel_area} * {efficiency} * august * 1e-3 AS august_solar_energy_production_mwh, "
        f"area * {effective_solar_panel_area} * {efficiency} * september * 1e-3 AS september_solar_energy_production_mwh, "
        f"area * {effective_solar_panel_area} * {efficiency} * october * 1e-3 AS october_solar_energy_production_mwh, "
        f"area * {effective_solar_panel_area} * {efficiency} * november * 1e-3 AS november_solar_energy_production_mwh, "
        f"area * {effective_solar_panel_area} * {efficiency} * december * 1e-3 AS december_solar_energy_production_mwh, "
        
        # Koszt budowy
        f"CASE "
        f"    WHEN area * {effective_solar_panel_area} * {efficiency} * 1e-3 < 0.5 THEN 3.51 * area * {effective_solar_panel_area} * {efficiency} * 1e-3 "
        f"    WHEN area * {effective_solar_panel_area} * {efficiency} * 1e-3 <= 1 THEN 2.63 * area * {effective_solar_panel_area} * {efficiency} * 1e-3 "
        f"    WHEN area * {effective_solar_panel_area} * {efficiency} * 1e-3 > 1 THEN 2.05 * area * {effective_solar_panel_area} * {efficiency} * 1e-3 "
        f"    ELSE 0 "
        f"END AS building_cost_mln_zl, "
        
        # Różnica w nasłonecznieniu
        f"((SELECT MAX(value) FROM (SELECT {sum_columns} AS value FROM {table_name}) subquery) - ({sum_columns})) AS solar_radiation_difference_kw "
        
        f"FROM {table_name}"
    )
    
    # Pobranie wyniku do pandas DataFrame
    result_df = pd.read_sql(query, conn)
    
    # Skopiowanie DataFrame przed dodaniem do słownika
    solar_radiation_photovoltaic_area_by_each_province[province] = result_df.copy()
    
    # Dodanie nowej kolumny z roczną produkcją energii
    solar_radiation_photovoltaic_area_by_each_province[province]['energy_production_by_year_mwh'] = solar_radiation_photovoltaic_area_by_each_province[province].loc[:, 'january_solar_energy_production_mwh':'december_solar_energy_production_mwh'].sum(axis=1)


In [ ]:
solar_radiation_photovoltaic_area_by_each_province['podlaskie']


## parametr sum narazie tego nie używam

In [ ]:
#create function which calculate value of road distance and distance to line 
#przyjmuje, że 1 to jest maksymalna odległość dla linii energetycznej to 500 m tak jest w wytycznych
#to samo robię dla drogi, parametr przyjmuje, że 1 km to jest 1 i rośnie, bo koszty są wysokie
# Druga pętla - tutaj zmiany
for province in provinces:    
    # Użycie danych z solar_radiation_photovoltaic_area_by_each_province, a nie z result_df
    df = solar_radiation_photovoltaic_area_by_each_province[province]
    
    # Obliczanie parametrów dla każdej prowincji
    parametr_line_distance = (df['line_distance_km'] - df['line_distance_km'].min()) / (df['line_distance_km'].max() - df['line_distance_km'].min())
    parametr_road_distance = (df['road_distance_km'] - df['road_distance_km'].min()) / (df['road_distance_km'].max() - df['road_distance_km'].min())

    df['road_distance_cost_parametr'] = parametr_road_distance
    df['line_distance_cost_parametr'] = parametr_line_distance

    df['solar_radiation_difference_parametr'] = (df['solar_radiation_difference_kw'] - df['solar_radiation_difference_kw'].min()) / (df['solar_radiation_difference_kw'].max() - df['solar_radiation_difference_kw'].min())
    
    # Kopiowanie danych dla modyfikacji
    temp_df = df.copy()
    
    # Zamiana wartości ujemnych w 'energy_production_by_year_mwh' i normalizacja
    temp_df['energy_production_by_year_mwh'] = -1 * temp_df['energy_production_by_year_mwh'] 
    df['solar_radiation_energy_parametr'] = (temp_df['energy_production_by_year_mwh'] - temp_df['energy_production_by_year_mwh'].min()) / (temp_df['energy_production_by_year_mwh'].max() - temp_df['energy_production_by_year_mwh'].min())

    # Zapisanie zaktualizowanych danych
    solar_radiation_photovoltaic_area_by_each_province[province] = df



In [ ]:
solar_radiation_photovoltaic_area_by_each_province['podlaskie']


## 11. Nadanie wag poszczególnym cechom obszarów pod budowę farm fotowoltaicznych, które wpływają na atrakcyjność obszaru

In [ ]:
import numpy as np
from scipy.optimize import minimize, Bounds

#waga_1 waga odleglosci od drogi
w_1 = 0.3

#waga_2 odleglosc od linii energetycznych
w_2 = 0.4

#waga_3 roznica w ilosci promieniowania 
w_3 = 0.1

#waga_4 ilosc energii, ktora mozna wyprodukowac w tym obszarze
w_4 = 0.2

#definiowanie funkcji celu
def objective(x, weights):
    
    return -weights[0] * x[0] - weights[1] * x[1] + weights[2] * x[2] + weights[3] * x[3]


initial_values = [0.5, 0.5, 0.5, 0.5]  

#granice 
bounds = Bounds([0, 0, 0, 0], [1, 1, 1, 1])  


weight_sets = [(w1, w2, w3, w4) for w1 in np.linspace(w_1, w_1 + 0.4, 5)
                                  for w2 in np.linspace(w_2, w_2 + 0.4, 5)
                                  for w3 in np.linspace(w_3, w_3 + 0.4, 5)
                                  for w4 in np.linspace(w_4, w_4 + 0.4, 5)]

## 12. Definiowanie funkcji celu obliczające wybrane cechy obszarów wpływające na ich atrakcyjnośc

In [ ]:
def calculate_objective(row, w1, w2, w3, w4):
    return -w1 * row['road_distance_cost_parametr'] - w2 * row['line_distance_cost_parametr'] \
           + w3 * row['solar_radiation_difference_parametr'] + w4 * row['solar_radiation_energy_parametr']

In [ ]:
for province in provinces:
    solar_radiation_photovoltaic_area_by_each_province[province]['type'] = 'solar'
    solar_radiation_photovoltaic_area_by_each_province[province]['province'] = province
    solar_radiation_photovoltaic_area_by_each_province[province]['objective_value'] = solar_radiation_photovoltaic_area_by_each_province[province].apply(calculate_objective, axis=1, args=(w_1, w_2, w_3, w_4))


In [ ]:
solar_radiation_photovoltaic_area_by_each_province['podlaskie']

## 13. Dane na temat rocznego zuzycia energii w wojewodztwach

In [ ]:
#zuzycie energii wg gus wg wojewodztw
energy_consumption_df = pd.read_csv('D:\GEOWORLDLOOK\OZE\PILOT\Data\ENERGY_CONSUMPTION\ENERGY_CONSUMPTION_BY_REGION.csv')

#wybranie danych z 2022
energy_consumption_df_2022 = energy_consumption_df[['wojewodztwa','2022']]

#usuniecie 1 wiersza z suma dla calej polski
energy_consumption_df_2022_by_region = energy_consumption_df_2022.drop(index = 0)



#zamiana gW na tW
energy_consumption_df_2022_by_region['2022'] = energy_consumption_df_2022_by_region['2022'] * 1e-3 
energy_consumption_df_2022_by_region['wojewodztwa'] = energy_consumption_df_2022_by_region['wojewodztwa'].str.replace('-', '_').str.lower()
energy_consumption_df_2022_by_region = energy_consumption_df_2022_by_region.rename(columns={'wojewodztwa': 'Province'})
energy_consumption_df_2022_by_region = energy_consumption_df_2022_by_region.rename(columns={'2022': 'energy_consumption_2022'})
energy_consumption_df_2022_by_region

## 14. Zbiorczy wykres przedstawiający zużycie roczne z podziałem na poszczególne województwa oraz teoretyczną produkcję energii elektrycznej i wiatrowej dla wszystkich wyselekcjonowanych obszarów

In [ ]:
# Łączenie danych na podstawie nazwy prowincji
data_combined = pd.merge(df_solar, df_wind, on='Province', suffixes=('_solar_energy_production_mwh', '_wind_energy_production_mwh'))

# Sumowanie danych miesięcznych dla energii słonecznej i wiatrowej
monthly_solar = data_combined.loc[:, 'january_solar_energy_production_mwh':'december_solar_energy_production_mwh'].sum(axis=1)
monthly_wind = data_combined.loc[:, 'january_wind_energy_production_mwh':'december_wind_energy_production_mwh'].sum(axis=1)

#obliczenie sumy produkcji energii rocznej dla województw ze względu na liczbę turbin
monthly_wind = monthly_wind# * df_wind['turbines_number'] 
# Tworzenie wykresu
fig, ax1 = plt.subplots(figsize=(12, 8))

# Wykres słupkowy dla powierzchni
ax1.bar(data_combined['Province'], data_combined['solar_effective_area'], color='b', label='Solar Effective Area', width=0.4, align='center')
ax1.bar(data_combined['Province'], data_combined['wind_area'], color='r', alpha=0.7, label='Wind Area', width=0.4, align='edge')
ax1.set_ylabel('Area (km²)')
ax1.set_title('Effective Area and Monthly Energy Production by Province')
tick_locations = range(len(data_combined['Province']))  # Positions for each tick
ax1.xaxis.set_major_locator(FixedLocator(tick_locations))
ax1.set_xticklabels(data_combined['Province'],rotation = 90)
ax1.legend(loc='upper left')

# Wykres liniowy dla produkcji energii
ax2 = ax1.twinx()
ax2.plot(data_combined['Province'], monthly_solar * 1e-6, 'g-', label='Total Solar Production')
ax2.plot(data_combined['Province'], monthly_wind * 1e-6, 'y-', label='Total Wind Production')
ax2.plot(energy_consumption_df_2022_by_region['Province'], energy_consumption_df_2022_by_region['energy_consumption_2022'], 'r-', label = 'demand for electricity in Poland in 2022')

ax2.set_ylabel('Total Energy Production (tW)')
ax2.legend(loc='upper right')

plt.show()

## 15. Suma produkcji solarnej i wiatrowej dla posczególnych województw


In [ ]:
data_combined['sum_production_solar'] = data_combined.loc[:, 'january_solar_energy_production_mwh':'december_solar_energy_production_mwh'].sum(axis=1)
data_combined['sum_production_wind'] = data_combined.loc[:, 'january_wind_energy_production_mwh':'december_wind_energy_production_mwh'].sum(axis=1)
data_combined = pd.merge(data_combined, energy_consumption_df_2022_by_region, on='Province')
data_combined

## 16. Wybranie dla każdego obszaru miesiąca z najmniejszą produkcją prądu

In [ ]:
data_combined['energy_consumption_2022'].sum()

In [ ]:
data_combined['min_value_solar'] = data_combined[['january_solar_energy_production_mwh',
       'february_solar_energy_production_mwh', 'march_solar_energy_production_mwh', 'april_solar_energy_production_mwh', 'may_solar_energy_production_mwh',
       'june_solar_energy_production_mwh', 'july_solar_energy_production_mwh', 'august_solar_energy_production_mwh', 'september_solar_energy_production_mwh',
       'october_solar_energy_production_mwh', 'november_solar_energy_production_mwh', 'december_solar_energy_production_mwh']].idxmin(axis=1) 
data_combined['min_value_wind'] = data_combined[['january_wind_energy_production_mwh', 'february_wind_energy_production_mwh', 'march_wind_energy_production_mwh',
       'april_wind_energy_production_mwh', 'may_wind_energy_production_mwh', 'june_wind_energy_production_mwh', 'july_wind_energy_production_mwh', 'august_wind_energy_production_mwh',
       'september_wind_energy_production_mwh', 'october_wind_energy_production_mwh', 'november_wind_energy_production_mwh', 'december_wind_energy_production_mwh']].idxmin(axis=1)
data_combined

## 17. Mając wybrane najmniejsze możliwości produkcyjne dla każdego województwa w najgorszym miesiacy szukamy takiej ilości, żeby zapewnić produkcję energii

In [ ]:
#filtracja danych, które spełniają warunek wytworzenia prądu z paneli solarnych w najgorszym możliwym okresie (grudzien) aby zapewnić miesięczną produkcję prądu
filtered_solar_radiation_photovoltaic_area_by_each_province = {}

for province in provinces:
    solar_radiation_photovoltaic_area_by_each_province[province] = solar_radiation_photovoltaic_area_by_each_province[province].sort_values(by = 'objective_value', ascending=False)
    
    #Wybranie wartości najmniejszej
    row = data_combined[data_combined['Province'] == province]
    column = row['min_value_solar'].iloc[0]
    solar_radiation_photovoltaic_area_by_each_province[province]['cumsum_tw'] = (solar_radiation_photovoltaic_area_by_each_province[province][column].cumsum()) * 1e-6
    
    #produkcja w tw najoptymalniejszych miejsc wybranych z wskaznikow
    # Tworzenie maski logicznej do momentu, gdy warunek jest spełniony
    
    row_min = energy_consumption_df_2022_by_region[energy_consumption_df_2022_by_region['Province'] == province]
    value_min = row['energy_consumption_2022'].iloc[0] / 12
    filtered_solar_radiation_photovoltaic_area_by_each_province[province]  = solar_radiation_photovoltaic_area_by_each_province[province][solar_radiation_photovoltaic_area_by_each_province[province]['cumsum_tw'] <value_min]


## 18. Przedstawienie na wykresie jak kształtuje się proukcja energii wiatrowej na przestrzenii roku dla każdego z województw

In [ ]:
df = pd.DataFrame(data = data_combined, columns = ['Province', 'january_wind_energy_production_mwh', 'february_wind_energy_production_mwh', 'march_wind_energy_production_mwh',
       'april_wind_energy_production_mwh', 'may_wind_energy_production_mwh', 'june_wind_energy_production_mwh', 'july_wind_energy_production_mwh', 'august_wind_energy_production_mwh',
       'september_wind_energy_production_mwh', 'october_wind_energy_production_mwh', 'november_wind_energy_production_mwh', 'december_wind_energy_production_mwh'])
# Ustawienie kolumny 'Province' jako indeks
df.set_index('Province', inplace=True)

# Przygotowanie danych do wykresu
months = df.columns  # Miesiące jako etykiety osi x
plt.figure(figsize=(12, 8))

# Rysowanie wykresu liniowego dla każdej prowincji
for province in df.index:
    plt.plot(months, df.loc[province], marker='o', label=province)

# Konfiguracja wykresu
plt.title('Produkcja energii w poszczególnych województwach')
plt.xlabel('Miesiąc')
plt.ylabel('Produkcja energii')
plt.legend(title='Województwo')
plt.xticks(rotation=45)  # Ułatwia czytanie nazw miesięcy
plt.grid(True)
plt.tight_layout()
plt.show()

## 19. Obliczenie dla każdego obszaru pod farmę wiatrową możliwości produkcyjnych oraz kosztów budowy

In [ ]:
month_days = {
    "january": 31,
    "february": 28,
    "march": 31,
    "april": 30,
    "may": 31,
    "june": 30,
    "july": 31,
    "august": 31,
    "september": 30,
    "october": 31,
    "november": 30,
    "december": 31
}

#dodanie atrybutow do obliczenia wskaznikow
#dystans od linii energetycznych
#koszt budowy 1 mW to 5-7 mln zl, przyjmujemy 6 wg  Polskiej Agencji Informacji i Inwestycji Zagranicznych.

wind_energy_production_by_each_province = {}
wind_farm_cost_by_mw = 6
for province in provinces:
    cumulative_df = pd.DataFrame()
    for month, days in month_days.items():
        table_name_wind = f"wind_speed_area_{province}"
        


        query = (
    f"SELECT "
    f"id, "
    f"area * 1e-6 as area_km2, "
    f"distance * 1e-3 as line_distance_km, "
    f"CASE "
    f"WHEN {month} > {minimum_wind_speed} THEN "
    f"(((({month} * 24 * {days}) / {maximum_wind_speed}) * {maximum_power_generate}) * (area * 1e-6 * {turbine_number_for_area})) "  # to są jednostki mW
    f"ELSE 0 "
    f"END AS {month} "
    f"FROM {table_name_wind} "
    f"WHERE area * 1e-6 > {(1 / turbine_number_for_area)}" # 1 km2 / 3 turbines for 1 km^2
)

        # result to pandas
        result_df = pd.read_sql(query, conn)

        if cumulative_df.empty:
            cumulative_df = result_df
        else:
            # Inaczej, złącz na podstawie kolumny 'id'
            cumulative_df = pd.merge(cumulative_df, result_df[['id', month]], on='id', how='left')

        wind_energy_production_by_each_province[province] = cumulative_df 

    wind_energy_production_by_each_province[province]['energy_production_by_year_mwh'] = wind_energy_production_by_each_province[province][['january','february', 'march', 'april','june','july','august','september','october', 'november','december']].sum(axis = 1)
    wind_energy_production_by_each_province[province]['building_cost_mln_zl'] = wind_energy_production_by_each_province[province]['energy_production_by_year_mwh'] * wind_farm_cost_by_mw
    

## 20. Obliczenie wskażników atrakcyjności obszaru dla farm wiatrowych

In [ ]:
for province in provinces:    
    wind_energy_production_by_each_province[province]
    
    #syntetyczny parametr, ktory wykazuje wzrost kosztów w zależności od odległości od drogi. Można to dowolnie parametryzować i obliczać realne koszty
    parametr_line_distance = (wind_energy_production_by_each_province[province]['line_distance_km'] - min(wind_energy_production_by_each_province[province]['line_distance_km'])) / (max(result_df['line_distance_km']) - min(result_df['line_distance_km']))


    wind_energy_production_by_each_province[province]['line_distance_cost_parametr'] = parametr_line_distance
    #normalizacja
    #poprawic wszystkie wg tego schematu lub napisać funkcję
    wind_energy_production_by_each_province[province]['difference_wind_production_for_km2'] =(wind_energy_production_by_each_province[province]['energy_production_by_year_mwh']/wind_energy_production_by_each_province[province]['area_km2'] - min((wind_energy_production_by_each_province[province]['energy_production_by_year_mwh']/wind_energy_production_by_each_province[province]['area_km2']) - wind_energy_production_by_each_province[province]['energy_production_by_year_mwh']/wind_energy_production_by_each_province[province]['area_km2']))/ (
    max((wind_energy_production_by_each_province[province]['energy_production_by_year_mwh']/wind_energy_production_by_each_province[province]['area_km2']) - min(wind_energy_production_by_each_province[province]['energy_production_by_year_mwh']/wind_energy_production_by_each_province[province]['area_km2'])))

#waga_1 waga odleglosci od drogi
w_1 = 0.3

#waga_2 odleglosc od linii energetycznych
w_2 = 0.4

#waga_3 roznica w ilosci promieniowania 
w_3 = 0.1

#waga_4 ilosc energii, ktora mozna wyprodukowac w tym obszarze
w_4 = 0.2


def calculate_objective_wind(row, w1, w2):
    return -w1 * row['line_distance_cost_parametr'] \
            +w2 * row['difference_wind_production_for_km2']

for province in provinces:
    wind_energy_production_by_each_province[province]['type'] = 'wind'
    wind_energy_production_by_each_province[province]['province'] = province
    wind_energy_production_by_each_province[province]['objective_value'] = wind_energy_production_by_each_province[province].apply(calculate_objective_wind, axis=1, args=(w_1, w_4))

## 21. Zebranie danych o farmach wiatrowych i solarnych w jedną zmienna

In [ ]:
df_area_solar_wind = {}
comb_all_area_solar_wind = pd.DataFrame()
for province in provinces:
    
    #filtered, bo to sa minimalne powierzchnie dla zapewnienia produkcji
    df_solar_area = filtered_solar_radiation_photovoltaic_area_by_each_province[province][['id','province','type', 'area_km2','objective_value','building_cost_mln_zl','energy_production_by_year_mwh',
        'january_solar_energy_production_mwh',
       'february_solar_energy_production_mwh',
       'march_solar_energy_production_mwh',
       'april_solar_energy_production_mwh', 'may_solar_energy_production_mwh',
       'june_solar_energy_production_mwh', 'july_solar_energy_production_mwh',
       'august_solar_energy_production_mwh',
       'september_solar_energy_production_mwh',
       'october_solar_energy_production_mwh',
       'november_solar_energy_production_mwh',
       'december_solar_energy_production_mwh']].rename(columns={
            'january_solar_energy_production_mwh': 'january',
            'february_solar_energy_production_mwh': 'february',
            'march_solar_energy_production_mwh': 'march',
            'april_solar_energy_production_mwh': 'april',
            'may_solar_energy_production_mwh': 'may',
            'june_solar_energy_production_mwh': 'june',
            'july_solar_energy_production_mwh': 'july',
            'august_solar_energy_production_mwh': 'august',
            'september_solar_energy_production_mwh': 'september',
            'october_solar_energy_production_mwh': 'october',
            'november_solar_energy_production_mwh': 'november',
            'december_solar_energy_production_mwh': 'december'
        })

    # Wybieranie odpowiednich kolumn dla wind #farmy wiatrowe jako wspomaganie
    wind_energy_production_by_each_province['dolnoslaskie'].columns
    df_wind_area = wind_energy_production_by_each_province[province][['id','province','type', 'area_km2','objective_value','building_cost_mln_zl','energy_production_by_year_mwh','january', 'february', 'march',
       'april', 'may', 'june', 'july', 'august', 'september', 'october',
       'november', 'december']]
    
    # Konkatenacja danych dla danej prowincji (łączenie wierszy)
    df_area_solar_wind[province] = pd.concat([df_solar_area, df_wind_area], ignore_index=True)
    comb_all_area_solar_wind = pd.concat([comb_all_area_solar_wind, df_area_solar_wind[province]], ignore_index=True)
    
#Posortowanie danych według atrakcyjności
comb_all_area_solar_wind = comb_all_area_solar_wind.sort_values(by = 'objective_value', ascending=False)

#Dodanie kolumny z całkowitą produkcją energii dla wybranych obszarów według atrakcyjności
comb_all_area_solar_wind['all_solar_wind_cumsum_twh'] = comb_all_area_solar_wind['energy_production_by_year_mwh'].cumsum() * 1e-6 #tWh


## 21. Wykres przedstawiający produkcję energii i zapotrzebowanie 

In [ ]:
# Tworzenie wykresu
fig, ax1 = plt.subplots(figsize=(12, 8))

for province in provinces:
    
    ax1.bar(province, df_area_solar_wind[province]['energy_production_by_year_mwh'].sum() * 1e-6) #twh
    
plt.xticks(rotation=90)  # Ułatwia czytanie nazw miesięcy
plt.grid(True)
plt.plot(data_combined['Province'],data_combined['energy_consumption_2022'], color = 'red')

plt.title('Wykres przedstawiający możliwości produkcyjne prądu dla każdego województwa w najgorszymi miesiącu pod względem możliwości wytwórczych')
plt.show()    

In [ ]:
df_area_solar_wind['podlaskie']

In [ ]:
comb_all_area_solar_wind['december'].sum()*1e-6

In [ ]:
for i in comb_all_area_solar_wind:
    print(i) 

In [ ]:
df = comb_all_area_solar_wind

# # Grupa na podstawie 'id' i sprawdzenie, czy w 'province' lub 'type' są różne wartości
# df_grouped = df.groupby('id').filter(lambda x: x['province'].nunique() > 1 or x['type'].nunique() > 1)
# 
# # Teraz usuniemy duplikaty 'id', zachowując tylko pierwsze wystąpienie
# df_unique = df_grouped.drop_duplicates(subset=[['id','province','type']])

In [ ]:
df_unique = df.drop_duplicates(subset=['id', 'province', 'type'],keep='first')

In [ ]:
df_unique

In [ ]:
duplicated_rows = comb_all_area_solar_wind.duplicated(subset=['id', 'province', 'type'])

# Wybranie tylko tych wierszy, które są duplikatami
df_duplicates = comb_all_area_solar_wind[duplicated_rows]
df_duplicates

In [ ]:
df_grouped = df.groupby('province')
df_grouped

In [ ]:
df_area_solar_wind['pomorskie']

In [ ]:
solar_radiation_photovoltaic_area_by_each_province['dolnoslaskie'][solar_radiation_photovoltaic_area_by_each_province['dolnoslaskie']['id']==4029]

In [ ]:
a = 0
b = 0
for province in provinces:
    a +=solar_radiation_photovoltaic_area_by_each_province[province].shape[0]
    b += wind_energy_production_by_each_province[province].shape[0]
    
print(a)
print(b)